In [38]:
from google.cloud import storage
import os

# 구글 클라우드 프로젝트 ID
project_id = "clear-talent-315601"
# 구글 클라우드 버킷 이름
bucket_name = "file_bucket_temp"
# 로컬 디렉토리 경로
local_directory_path = "/Users/minjikim/engine-git/crawler-disclosure/files"

# 인증서 파일 위치 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/minjikim/engine-git/crawler-disclosure/saas-engine-storage.json"

In [39]:
# 클라이언트 초기화
storage_client = storage.Client(project=project_id)

# 버킷 가져오기
bucket = storage_client.get_bucket(bucket_name)

completed=[]
error_files=[]
# 로컬 디렉토리에서 .pdf 파일 검색 후 버킷에 업로드
pdf_files = [f for f in os.listdir(local_directory_path) if f.endswith("약관.pdf")]

for idx, filename in enumerate(pdf_files):
    file_path = os.path.join(local_directory_path, filename)
    try:
        blob = bucket.blob(filename)
        blob.upload_from_filename(file_path)
        completed.append(idx)
        print(f"{filename} uploaded to {bucket_name}.")
    except:
        error_files.append(idx)
        print("ERROR: ", idx, filename)


True_메리츠화재_상해보험_실손의료보험_2017-01-01_2017-03-31_약관.pdf uploaded to file_bucket_temp.
False_메리츠화재_상해보험_신마이라이프II_1999-05-01_2000-05-01_약관.pdf uploaded to file_bucket_temp.
False_메리츠화재_질병보험_무배당메리츠올바른간편보험2301간편심사형_2023-01-01_2023-02-06_약관.pdf uploaded to file_bucket_temp.
True_메리츠화재_생활보험_해외장기체류실손의료보험_2018-11-09_2018-12-31_약관.pdf uploaded to file_bucket_temp.
False_메리츠화재_어린이보험_무배당내Mom같은어린이보험2101해지환급금미지급형_2021-01-21_2021-02-17_약관.pdf uploaded to file_bucket_temp.
True_메리츠화재_화재_재물_비용보험_중장비안전보험_2019-06-01_2019-12-31_약관.pdf uploaded to file_bucket_temp.
True_메리츠화재_화재_재물_비용보험_중장비안전보험_2020-01-01_2020-04-15_약관.pdf uploaded to file_bucket_temp.
False_메리츠화재_어린이보험_무배당내Mom같은베이비보험2303_2023-03-07_2023-04-11_약관.pdf uploaded to file_bucket_temp.
False_메리츠화재_상해보험_무배당메리츠올인원라이프보장보험18012종_2018-01-01_2018-04-01_약관.pdf uploaded to file_bucket_temp.
True_메리츠화재_생활보험_전문직해외장기체류실손의료보험_2016-11-28_2016-12-31_약관.pdf uploaded to file_bucket_temp.
True_메리츠화재_상해보험_다모아상해보험_2016-01-01_2016-11-27_약관.pdf uploaded to file_bucket_

In [45]:
print("총 개수: ",len(pdf_files), "에러 개수: ",len(error_files), "완료된 개수: ", len(completed))

총 개수:  4156 에러 개수:  0 완료된 개수:  4156


In [89]:
import pandas as pd

fileUrl= pd.read_excel('error.xlsx')
meritz=fileUrl[fileUrl['name']=='메리츠화재']
meritz['url']= meritz['url'].str.replace(' ','_')
meritz['url']= meritz['url'].str.replace('(','')
meritz['url']= meritz['url'].str.replace(')','')
meritz['url']= meritz['url'].str.replace('&','')
meritz['url']= meritz['url'].str.replace('!','')
meritz['url']= meritz['url'].str.replace(',','')
meritz['url']= meritz['url'].str.replace(',','')
meritz['url']= meritz['url'].str.replace('∙','')
meritz


/var/folders/zr/zcxyh13d1qj0cly9bgl4gbw80000gn/T/ipykernel_19653/2153534388.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  meritz['url']= meritz['url'].str.replace('(','')
/var/folders/zr/zcxyh13d1qj0cly9bgl4gbw80000gn/T/ipykernel_19653/2153534388.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  meritz['url']= meritz['url'].str.replace(')','')


,name,name.1,start_date,end_date,url
0,메리츠화재,무배당335오!간편건강보험2302,2023-03-07,NaN,True_메리츠화재_상해보험_무배당335오간편건강보험2302_2023-03-07_n...
1,메리츠화재,무배당335오!간편건강보험(Ⅱ)2303,2023-03-07,NaN,True_메리츠화재_상해보험_무배당335오간편건강보험Ⅱ2303_2023-03-07_...
2,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2022-01-06,2022-03-31,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2022-01-...
3,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2022-04-01,2022-04-27,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2022-04-...
4,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2022-04-28,2022-06-30,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2022-04-...
5,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2022-07-01,2022-12-31,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2022-07-...
6,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2023-01-01,NaN,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2023-01-...
7,메리츠화재,교통사고및자전거사고처리지원금보장보험금지급관련특별약관,2022-10-17,2022-12-31,True_메리츠화재_제도성특약_교통사고및자전거사고처리지원금보장보험금지급관련특별약관_...
8,메리츠화재,"교통사고처리지원금(스쿨존,어린이42일미만치료)추가보장특별약관",2020-06-08,NaN,True_메리츠화재_제도성특약_교통사고처리지원금스쿨존어린이42일미만치료추가보장특별약...
9,메리츠화재,단체취급특별약관,2023-01-01,NaN,True_메리츠화재_제도성특약_단체취급특별약관_2023-01-01_null_약관.pdf


In [90]:
# GCS 클라이언트 초기화
client = storage.Client()

# 버킷 객체 가져오기
bucket = client.get_bucket(bucket_name)

new_df = pd.DataFrame(columns=["filename"])
# meritz 데이터프레임 순회하며 public URL 가져오기
for idx, row in meritz.iterrows():
    # '메리츠화재' 텍스트가 포함된 경우에만 처리
    if '메리츠화재' in row['name']:
        # GCS 객체 존재 여부 확인
        blob = bucket.blob(row['url'])
        if blob.exists():
            # public URL 가져오기
            storage_url = blob.public_url
            # URL 업데이트
            meritz.loc[idx, 'url'] = storage_url
        else:
            new_df = new_df.append({"filename": filename}, ignore_index=True)

In [80]:
new_df

,filename
0,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
1,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
2,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
3,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
4,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
...,...
134,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
135,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
136,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...
137,False_메리츠화재_상해보험_무배당메리츠The간편한건강보험Ⅰ19041종_2019-...


In [71]:
# GCS 클라이언트 초기화
client = storage.Client()

# 버킷 객체 가져오기
bucket = client.get_bucket(bucket_name)

# new_df = pd.DataFrame(columns=["filename"])
# fileUrl 데이터프레임 순회하며 public URL 가져오기
for idx, row in error.iterrows():
    # '메리츠화재' 텍스트가 포함된 경우에만 처리
    if '메리츠화재' in row['name']:
        # GCS 객체 존재 여부 확인
        blob = bucket.blob(row['url'])
        if blob.exists():
            # public URL 가져오기
            storage_url = blob.public_url
            # URL 업데이트
            error.loc[idx, 'url'] = storage_url
        # else:
            # new_df = new_df.append({"filename": filename}, ignore_index=True)

In [72]:
error

,name,name.1,start_date,end_date,url
17296,메리츠화재,(신)노후안심,1997-07-01,1999-05-01,False_메리츠화재_연금저축보험_(신)노후안심_1997-07-01_1999-05-...
17297,메리츠화재,(신)마이라이프,1997-07-01,1999-04-30,False_메리츠화재_상해보험_(신)마이라이프_1997-07-01_1999-04-3...
17298,메리츠화재,(신)실버그린,1997-07-01,1999-05-01,False_메리츠화재_연금저축보험_(신)실버그린_1997-07-01_1999-05-...
17299,메리츠화재,(신)저축형빅화이브상해,1997-07-01,1999-04-30,False_메리츠화재_상해보험_(신)저축형빅화이브상해_1997-07-01_1999-...
17300,메리츠화재,(신)홈런인생,1997-07-01,1999-02-01,False_메리츠화재_연금저축보험_(신)홈런인생_1997-07-01_1999-02-...
17301,메리츠화재,「무배당웰스라이프보험」자유설계자동갱신변경적용특별약관,2017-02-01,NaN,False_메리츠화재_제도성특약_「무배당웰스라이프보험」자유설계자동갱신변경적용특별약관...
17302,메리츠화재,21세기가정종합보험(II),1999-05-01,2000-05-01,False_메리츠화재_화재_재물_비용보험_21세기가정종합보험(II)_1999-05-...
17303,메리츠화재,21세기운전자상해보험(II),1999-05-01,2000-04-30,False_메리츠화재_운전자보험_21세기운전자상해보험(II)_1999-05-01_2...
17304,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2022-01-06,2022-03-31,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2022-01-...
17305,메리츠화재,4세대실손의료비계약전환특별할인특별약관,2022-04-01,2022-04-27,True_메리츠화재_제도성특약_4세대실손의료비계약전환특별할인특별약관_2022-04-...


In [81]:
new_df.to_excel('error.xlsx',index=False)

In [66]:
len(fileUrl[fileUrl['url'].str.contains('www.meritz')])

41

In [91]:
meritz.to_excel('test2.xlsx',index=False)

In [44]:
# for file_name in os.listdir(local_directory_path):
#     if '약관_' in file_name:
#         file_path = os.path.join(local_directory_path, file_name)
#         os.remove(file_path)

In [47]:
tt_files = [f for f in os.listdir(local_directory_path) if f.endswith(".pdf")]

len(tt_files)

4156

In [29]:
# for file_name in os.listdir(local_directory_path):
#     if '요약서' in file_name:
#         file_path = os.path.join(local_directory_path, file_name)
#         os.remove(file_path)

In [ ]:
# 'meritz' 텍스트가 포함된 경우 해당 row 삭제
    fileUrl = fileUrl[fileUrl['url'].str.contains('meritz')]